<a href="https://colab.research.google.com/github/jrmaza/machine-learning/blob/main/Avalon%20fingerprints%20for%20predictive%20modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install rdkit-pypi==2022.3.1.1

#### In the previous [molecular fingerprints video](https://www.youtube.com/watch?v=-oHqQBUyrQ0), we saw how to generate molecular fingerprints and use them for structure search.

#### In this notebook, I will use Avalon fingerprints to develop predictive models for molecular property (energy gap) and reactions outcome (yield and enantioselectivity).
* HOMO-LUMO energy gap of organic compouds
* Yield Prediction of C–N cross coupling reactions
* Catalytic enantioselectivity of thiol-imine reactions

<div>
    <img src = "Fingerprints_predictive.JPG", width="600">
</div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
#-------------------------------------------------------
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.DataStructs.cDataStructs import ExplicitBitVect
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm
#--------------------------------------------------------
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import time
from sklearn.model_selection import ShuffleSplit, cross_validate,train_test_split
#--------------------------------------------------------
from lightgbm import LGBMRegressor

### Dataset obtained [Sandfort et al.](https://zivgitlab.uni-muenster.de/m_kueh11/fp-dm-tool).


<div style= " border-style:dotted;text-align:left;color:blue;font-size:25px;border-radius: 10px;padding: 20px 20px;">
1. HOMO-LUMO energy gap of molecules. 

</div>


In [ ]:
# Reference for the datasets: https://zivgitlab.uni-muenster.de/m_kueh11/fp-dm-tool

dataset = pd.read_csv("https://raw.githubusercontent.com/gashawmg/Avalon-fingerprints-for-machine-learning/main/Orbital_Energies_input_data.csv")
dataset.head()

In [ ]:
dataset.shape

In [ ]:
# Include 2D structure of molecules
PandasTools.AddMoleculeColumnToFrame(dataset,'SMILES','Stucture',includeFingerprints=True)

In [ ]:
dataset.head()

<div style= " border-style:dotted;text-align:left;color:purple;font-size:20px;border-radius: 10px;padding: 20px 20px;"> 
    
* Function defination for generating Avalon molecular fingerprints using RDKit
</div>

In [ ]:
def generate_AVfpts(data):
    
    # Store fingerprints using empty lists
    
    Avalon_fpts = []
    
    # Here no need to generate mol files
    # We can directly use the structure column of the data frame
    # mols = [Chem.MolFromSmiles(x) for x in data if x is not None]
    for mol in tqdm(data):
      
               
        #=============================
        # calculate Avalon fingerprints
        #=============================
        avfpts = pyAvalonTools.GetAvalonFP(mol, nBits=4096)
        Avalon_fpts.append(avfpts)
        
        
    #=====append=========================
    # return the lists as numpy array 
    #====================================
    return np.array(Avalon_fpts)

<div style= "border-style:dotted;text-align:left;color:darkblue;font-size:20px;border-radius: 10px;padding: 20px 20px;"> 
    
* Function call to generate Avalon molecular fingerprints of compounds that have HOMO-LUMO energy gap.
</div>

In [ ]:
# Function call
Avalon_fpts = generate_AVfpts(dataset['Stucture'])

### Put the fingerprints in dataframe

In [ ]:
# Put Avalon fingerprints into Dataframe
Avalon_fingerprints = pd.DataFrame(Avalon_fpts, columns=['AvFp_{}'.format(i + 1) 
                                                        for i in range(Avalon_fpts.shape[1])])

In [ ]:
Avalon_fingerprints.head()

In [ ]:
# Instantiate the models
lgbm_av = LGBMRegressor(n_estimators=800, random_state=42)
model_rf = RandomForestRegressor(random_state=42)

<div style= "border-style:dotted;text-align:left;color:darkblue;font-size:20px;border-radius: 10px;padding: 20px 20px;"> 
10 fold cross-validation using 4,096 bits of Avalon fingerprints
</div>

In [ ]:
# start time
start = time.time()

# tenfold cross-validation
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
scoring = ['r2','neg_mean_absolute_error']

scores_val = cross_validate(lgbm_av, Avalon_fingerprints,  dataset.Energygap, scoring=scoring, cv=cv)

print(scores_val)

time.sleep(3)
# get the end time
end = time.time()

# get the execution time
total_time = end - start
print('Total time required:', round(total_time/60, 2), 'min')

In [ ]:
# R^2 of coefficient of determination of tenfold cross validation
scores_val['test_r2']

In [ ]:
# Mean absolute error (MAE) of the tenfold cross validation values
-scores_val['test_neg_mean_absolute_error']

In [ ]:
# Mean values of tenfold cross-validation values of R^2 and MAE 
# Using lightGBM

print("R^2 :", round(np.mean(scores_val['test_r2']), 2))
print("MAE :", round(np.mean(-scores_val['test_neg_mean_absolute_error']), 2))

Compared with published result: [Sandfort et al's](https://www.sciencedirect.com/science/article/pii/S2451929420300851) work.
<div>
    <img src="https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/HUMO_LUMO.JPG?raw=1", align = "left", width = "500">
    </div

[ See supporting information of the paper ](https://zivgitlab.uni-muenster.de/m_kueh11/fp-dm-tool)
.Note this information is found on page 5 in the supporting information 

<div>
<img src= "Feature Preprocessing.JPG",align = "left", width = "600" />
 </div>

## 1.  Train-test data split  for Avalon fingerprints

In [ ]:
X_train_av, X_test_av, y_train_av, y_test_av = train_test_split(Avalon_fingerprints,
                                                    dataset.Energygap,test_size=0.3,random_state=42)

In [ ]:
# Model training and test
model_av = lgbm_av.fit(X_train_av,y_train_av)
predict_av = model_av.predict(X_test_av)

In [ ]:
# Model performance using MAE
MAE_av = mean_absolute_error(y_test_av, predict_av)
print('MAE:', round(MAE_av,2))

MAE: 5.79


In [ ]:
# R^2 (coefficient of determination) regression score function: 
R2_av =r2_score(y_test_av, predict_av)
print('R^2:', round(R2_av,2))

R^2: 0.91


In [ ]:
sns.regplot(x=predict_av, y=y_test_av,line_kws={"lw":2,'ls':'--','color':'black',"alpha":0.7})
plt.xlabel('Predicted Energy gap', color='blue')
plt.ylabel('Observed Energy gap', color ='blue')
plt.title("Avalon fpts HOMO-LUMO Energy Gap Prediction", color='red')
plt.grid(alpha=0.2)
R2_av = mpatches.Patch(label="R2={:04.2f}".format(R2_av))
MAE_av = mpatches.Patch(label="MAE={:04.2f}".format(MAE_av))
plt.legend(handles=[R2_av, MAE_av])

# 2. **Yield Prediction: C–N cross coupling reactions**


Reference:[Ahneman et al](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C14&q=Predicting+reaction+performance+in+C%E2%80%93N+cross-coupling+using+machine+learning&btnG=).
and [for supplemental information](https://www.science.org/doi/abs/10.1126/science.aar5169)

<div>
    <img src="https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/buchwald.JPG?raw=1" width="700",align ="left"/>
</div>

* Used 1536 plate (3 plates => 3 x 1536 (32 x 48)  = 4608)
* Removed control reactions and additive 7 
* 3,960 reactions were used for machine learning.
* Determined yield UPLC, di-tert-butylbiphenyl as internal standard (probably using response factor)
### Computed Descriptors (120)
* B3LYP/6-31G*
### Calculated various descriptors: 
* electrostatic charge and NMR shift.
* molecular volume, 
* surface area, ovality, molecular weight, HOMO,  
* LUMO, electronegativity, hardness, dipole moment,
* frequency and intensity.

### Generated 120 computed descriptors
* Additive Descriptors (19)
* Aryl Halide Descriptors ( 27)
* Base Descriptors (10)
* Ligand Descriptors (64)

<div>
    <img src="https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/Buchwald_RF.JPG?raw=1" width="600",align ="left"/>
</div>


There was an excel file that I downloaded converted it to csv file from this reference. [Sandfort et al](https://zivgitlab.uni-muenster.de/m_kueh11/fp-dm-tool). 

In [ ]:
# Read data frame
data = pd.read_csv('https://raw.githubusercontent.com/gashawmg/Avalon-fingerprints-for-machine-learning/main/Buchwald_yield_data.csv')
data

In [ ]:
def Calculate_Avfps(data,nBits):
    
    # Store fingerprints using empty lists
    Avalon_fpts = []
           
    mols = [Chem.MolFromSmiles(x) for x in data if x is not None]
    for mol in tqdm(mols):
        
               
        #=============================
        # calculate Avalon fingerprints
        #=============================
        avfpts = pyAvalonTools.GetAvalonFP(mol, nBits=nBits)
        Avalon_fpts.append(avfpts)
        
       
    #=================================
    # return the lists as numpy array 
    #=================================
    return np.array(Avalon_fpts)

## Calculate fingerprints for reactants

In [ ]:
ligand_fpts =  Calculate_Avfps(data['Ligand'], 2048) # ligands are relatively bigger than the other reagents
additive_fpts =  Calculate_Avfps(data['Additive'], 1024)
base_fpts =  Calculate_Avfps(data['Base'], 1024)
Aryl_fpts =  Calculate_Avfps(data['Aryl halide'], 1024)

## Concatenate fingerprints of starting materials
Similar to [Sandfort et al's](https://www.sciencedirect.com/science/article/pii/S2451929420300851) work.

In [ ]:
# concatenate the fingerprints of the four variables 
# note that the order of concatenation should not matter

concatenated = np.concatenate([ligand_fpts, additive_fpts, base_fpts, Aryl_fpts], axis=1)
concatenated.shape

(3955, 5120)

In [ ]:
# Put them in a data frame
concatenated_df = pd.DataFrame(concatenated,columns=['AvFp_{}'.format(i + 1) for i in range(concatenated.shape[1])])
concatenated_df.head()

In [ ]:
# start time
start = time.time()

# from Sklearn -shuffle and split data 
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

scoring = ['r2','neg_root_mean_squared_error'] 

scores_CN = cross_validate(lgbm_av, concatenated_df,  data.Output, scoring=scoring, cv=cv)


print(scores_CN)
# wait for 3 seconds

time.sleep(3)
# get the end time
end = time.time()

# get the execution time
total_time = end - start
print('Total time required:', round(total_time/60, 2), 'min') # takes about 0.27 min

### Notes: Avalon fingerprints used,  for each reagent 1024 bits used except the ligands. The ligands are relatively bigger, and 2048 bits were used.

In [ ]:
scores_CN['test_r2']

In [ ]:
# Mean values of tenfold cross-validation values of R^2 and RMSE

print("R^2 :", round(np.mean(scores_CN['test_r2']), 2))
print("RMSE :", round(np.mean(-scores_CN['test_neg_root_mean_squared_error']), 2))

[ Ahneman et al.](https://doylecpanel.cpaneldev.princeton.edu/wp-content/uploads/2020/07/43-Predicting-Reaction-Performance-in-C-N-Cross-Coupling-Using-Machine-Learning.pdf) <br>
[Sandfort et al.](https://www.sciencedirect.com/science/article/pii/S2451929420300851)

<table>  
    <tr>
     <td><img src = "Buchwald_performance.JPG", align = "left", width = "400"/></td>
     <td><img src="https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/Buchwald_RF.JPG?raw=1" width="600",align ="left"/>
     </td>
    </tr>
</table>


[ See supporting information of the paper](https://zivgitlab.uni-muenster.de/m_kueh11/fp-dm-tool)


<div>
<img src= "Feature Preprocessing.JPG",align = "left", width = "600" />
 </div>

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(concatenated_df, data.Output, 
                                                test_size = 0.3, shuffle=True,random_state=42)

In [ ]:
# Model training and prediction
# All the features are used
model_all_feature = LGBMRegressor(n_estimators=800,random_state=42)
model_all_feature.fit(X_train,y_train)
predicted_yield_all_feature = model_all_feature.predict(X_test)

In [ ]:
# Calculate root-mean-squared error (RMSE) & coefficient of determination
RMSE_all =np.sqrt(mean_squared_error(y_test, predicted_yield_all_feature))
R2_all = r2_score(y_test, predicted_yield_all_feature)
print('RMSE: ',round(RMSE_all,2))
print('R2: ', round(R2_all,2))

In [ ]:
# Plot predicted and observed yields
plt.figure(figsize=(5,5))
sns.regplot(x=predicted_yield_all_feature, y=y_test,line_kws={"lw":2,'ls':'--','color':'black',"alpha":0.7})
plt.xlabel('Predicted Yield', color='blue')
plt.ylabel('Observed Yield', color ='blue')
plt.xlim(0,100)
plt.ylim(0,100)
plt.title("Yield prediction for C-N cross-coupling", color='red')
plt.grid(alpha=0.2)
R2_all = mpatches.Patch(label="R2={:04.2f}".format(R2_all))
RMSE_all = mpatches.Patch(label="RMSE={:4.2f}".format(RMSE_all))
plt.legend(handles=[R2_all, RMSE_all])

# 3. **Prediction of Enantioselectivity**


<div>
    <img src = "Enantioselective.JPG", width = "500">
  </div>
    
  Reference 1: [Zahrt et al ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6417887/)  
  Reference 2: [Sandfort et al ](https://www.sciencedirect.com/science/article/pii/S2451929420300851) 

In [ ]:
Enantioselective = pd.read_csv('https://raw.githubusercontent.com/gashawmg/Avalon-fingerprints-for-machine-learning/main/Denmark_input_data.csv')
Enantioselective.head()

,Catalyst,Imine,Thiol,Output
0,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC=C2,SC1=CC=CC=C1,1.179891
1,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC=C2,CCS,0.501759
2,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC=C2,SC1CCCCC1,0.650584
3,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC=C2,SC1=CC=C(OC)C=C1,1.238109
4,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,SC1=CC=CC=C1,1.179891


In [ ]:
# Calculate fingerprints for reactants
catalyst_fpts =  Calculate_Avfps(Enantioselective['Catalyst'], 2048)
imine_fpts =  Calculate_Avfps(Enantioselective['Imine'], 1024)
thiol_fpts =  Calculate_Avfps(Enantioselective['Thiol'], 1024)

In [ ]:
# Concatenate fingerprints of starting materials
concatenated_fpts = np.concatenate([catalyst_fpts,imine_fpts,thiol_fpts],axis=1)
concatenated_fpts.shape

(1075, 4096)

In [ ]:
# Put them in a dataframe
Enan_concatenated_df = pd.DataFrame(concatenated_fpts,columns=['AvFp_{}'.format(i + 1) for i in range(concatenated_fpts.shape[1])])
Enan_concatenated_df

In [ ]:
Enan_concatenated_df.shape

In [ ]:
# Random forest model for enantioselective reaction
model_rf_enan = RandomForestRegressor(random_state=42)

# start time
start = time.time()

# from Sklearn -shuffle and split data 
cv = ShuffleSplit(n_splits=10, test_size = 0.441, random_state=42)

scoring = ['r2','neg_mean_absolute_error']
scores_enan = cross_validate(model_rf_enan, Enan_concatenated_df,  Enantioselective.Output, scoring=scoring, cv=cv)


print(scores_enan)
# wait for 3 seconds

time.sleep(3)
# get the end time
end = time.time()

# get the execution time
total_time = end - start
print('Total time required:', round(total_time, 2), 'seconds') # takes about 19 seconds

In [ ]:
# Mean values of tenfold cross-validation values of R^2 and MAE

print("R^2 :", round(np.mean(scores_enan['test_r2']), 3))
print("MAE :", round(np.mean(-scores_enan['test_neg_mean_absolute_error']), 3))

 [Sandfort et al ](https://www.sciencedirect.com/science/article/pii/S2451929420300851) 
<div>
   <img src = "Enantioselective_graph.JPG",align ="left", width = "400"/>
 </div>


In [ ]:
# Train-test split
X_train_enan, X_test_enan, y_train_enan, y_test_enan = train_test_split(Enan_concatenated_df, Enantioselective.Output, 
                                                test_size = 0.441, shuffle=True,random_state=42)

In [ ]:
# Model training and prediction
# All the features are used
model_enan = RandomForestRegressor(random_state=42)
model_enan.fit(X_train_enan,y_train_enan)
predicted_yield_enan = model_enan.predict(X_test_enan)

In [ ]:
# Calculate root-mean-squared error (RMSE) & coefficient of determination
# Model performance using MAE
MAE_enan = mean_absolute_error(y_test_enan, predicted_yield_enan)
print('MAE:', round(MAE_enan,3))
R2_enan = r2_score(y_test_enan, predicted_yield_enan)
print('R2: ', round(R2_enan,2))

In [ ]:
# Plot predicted and observed yields
plt.figure(figsize=(5,5))
sns.regplot(x=predicted_yield_enan, y=y_test_enan,line_kws={"lw":2,'ls':'--','color':'black',"alpha":0.7})
plt.xlabel('Predicted (kcal/mol)', color='blue')
plt.ylabel('Observed (kcal/mol)', color ='blue')
plt.title("Random Forest for Enantioselectivity", color='red')
plt.grid(alpha=0.2)
R2_enan = mpatches.Patch(label="R2={:04.2f}".format(R2_enan))
MAE_enan = mpatches.Patch(label="MAE={:4.3f}".format(MAE_enan))
plt.legend(handles=[R2_enan, MAE_enan])

<div style="text-align:left;color:blue;font-size:25px;border-radius: 10px;padding: 20px 20px;"><strong>Practice Problems</strong></div>

1. Use any of the other molecular fingerprints that we calculated in the previous [video](https://www.youtube.com/watch?v=-oHqQBUyrQ0), and calculate the molecular fingerprints of each of the three datasets that we used in this video.
2. Use a Random Forest model and apply a 10-fold cross-validation with 70% of the data for the training and 30% for testing. 
3. Compare your results with the results of Avalon fingerprints  

<div style="text-align:left;color:green;font-size:25px;border-radius: 10px;padding: 20px 20px;"><strong>Summary:</strong></div>
Using Avalon fingerprints as an example, we have developed predictive models for three different types of chemistry problems, HOMO-LUMO energy gap, yield and enantioselective reactions. Hopefully, in the next video, I will show you how to combine results obtained from machine learning and molecular docking to identify potentially active compounds. Stay tuned!